In [ ]:
from flask import Flask,request,jsonify
from flask_cors import CORS
import base64
import cv2
import mediapipe as mp
import numpy as np

app = Flask(_name_)
CORS(app)
counter = int(0)
stage = None

def reset_vals():
     global counter
     global stage
     counter = int(0)
     stage= None


@app.route('/kurls', methods=['POST'])
def start_camera():
    
    global counter
    global stage
   
    target = int(request.args.get('target'))
    reset = int(request.args.get('reset'))
    if reset == 1:
         reset_vals()
    
    mp_drawing = mp.solutions.drawing_utils
    mp_pose = mp.solutions.pose
    
    def calculate_angle(a, b, c):
        a = np.array(a)  
        b = np.array(b)  
        c = np.array(c) 
    
        radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
        angle = np.abs(radians * 180.0 / np.pi)
    
        if angle > 180.0:
            angle = 360 - angle
    
        return angle
    
   
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        
                image_bytes = base64.b64decode(request.json['image'])
                fr = np.frombuffer(image_bytes, np.uint8)
                frame = cv2.imdecode(fr, cv2.IMREAD_COLOR)
        
               
                imager = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                imager.flags.writeable = False
        
                # print(frame)
                # print(frame.shape)
        
                results = pose.process(imager)
        
                imager.flags.writeable = True
                imager = cv2.cvtColor(imager, cv2.COLOR_RGB2BGR)
        
                try:
                    # print(counter)
                    landmarks = results.pose_landmarks.landmark
        
                    rshoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                    relbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                    rwrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
        
                    angle = calculate_angle(rshoulder, relbow, rwrist)
        
                    # cv2.putText(imager, str(angle),
                    #             tuple(np.multiply(elbow, [720, 480]).astype(int)),
                    #             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

                    
                    v1 = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].visibility
                    v2 = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].visibility
                    v3 = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].visibility
                    
                    if v1 >= 0.40 and v2 >= 0.40 and v3 >= 0.40:
                        if angle > 120:
                            stage = "down"
                        if angle < 50 and stage == 'down':
                            stage = "up"
                            counter += 1
                    else:
                        cv2.putText(imager, 'Right elbow not visible', (320, 60), 2, 2, (0, 0, 0), 2, cv2.LINE_AA)


                    
                    lshoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                    lelbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                    lwrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
        
                    angle = calculate_angle(lshoulder, lelbow, lwrist)
        
                    # cv2.putText(imager, str(angle),
                    #             tuple(np.multiply(elbow, [720, 480]).astype(int)),
                    #             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

                    
                    l1 = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].visibility
                    l2 = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility
                    l3 = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].visibility
                    
                    if l1 >= 0.40 and l2 >= 0.40 and l3 >= 0.40:
                        if angle > 120:
                            stage = "down"
                        if angle < 50 and stage == 'down':
                            stage = "up"
                            counter += 1
                    else:
                        cv2.putText(imager, 'Left elbow not visible', (320, 60), 2, 2, (0, 0, 0), 2, cv2.LINE_AA)


        
                except:
                    pass
        
                cv2.rectangle(imager, (0, 0), (300, 75), (245, 117, 16), -1)
        
                cv2.putText(imager, 'REPS', (15, 12),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(imager, str(counter),
                            (10, 65),
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        
                cv2.putText(imager, 'STAGE', (160, 12),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(imager, stage,
                            (120, 65),
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        
                mp_drawing.draw_landmarks(imager, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(0, 0, 0), thickness=2, circle_radius=1),
                                        mp_drawing.DrawingSpec(color=(255, 255, 0), thickness=1, circle_radius=2))
        
                x=int(0)
                if(counter == target):
                     x=1
                _, buffer = cv2.imencode('.jpg', imager)
                processed_image_data = base64.b64encode(buffer).decode('utf-8')
                cv2.destroyAllWindows()
                return jsonify({'processed_image': processed_image_data,'isTarget':x})
    
        
    return 'Camera started'







































if _name_ == "_main_":
    app.run(host='0.0.0.0', port=5000)